In [8]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Annotated, TypedDict, Literal
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv
from operator import add

In [2]:
load_dotenv(dotenv_path="../.env")

True

In [4]:
generator_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=1.2)
evaluator_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)
optimizer_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.8)

In [7]:
class TweetEvaluation(BaseModel):

    evaluation: Literal["approved", "rejected"] = Field(..., description="The evaluation of the tweet")
    feedback: str = Field(..., description="Feedback on the tweet")

structured_evaluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)

In [9]:
class TweetState(TypedDict):

    topic: str
    tweet: str
    evaluation: Literal["approved", "rejected"]
    feedback: str
    iteration: int
    max_iterations: int

    tweet_history: Annotated[list[str], add]
    feedback_history: Annotated[list[str], add]

In [10]:
def generate_tweet(state: TweetState) -> TweetState:

    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
    ]

    response = generator_llm.invoke(messages).content

    return {"tweet": response, "tweet_history": [response]}

def evaluate_tweet(state: TweetState):

    messages = [
    SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
    HumanMessage(content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality – Is this fresh, or have you seen it a hundred times before?  
2. Humor – Did it genuinely make you smile, laugh, or chuckle?  
3. Punchiness – Is it short, sharp, and scroll-stopping?  
4. Virality Potential – Would people retweet or share it?  
5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"  
- feedback: One paragraph explaining the strengths and weaknesses 
""")
]

    response = structured_evaluator_llm.invoke(messages)

    return {'evaluation':response.evaluation, 'feedback': response.feedback, 'feedback_history': [response.feedback]}

def optimize_tweet(state: TweetState):

    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
""")
    ]

    response = optimizer_llm.invoke(messages).content
    iteration = state['iteration'] + 1

    return {'tweet': response, 'iteration': iteration, 'tweet_history': [response]}

def route_evaluation(state: TweetState):

    if state['evaluation'] == "approved" or state['iteration'] == state['max_iterations']:
        return "approved"
    
    else:
        return "needs_improvement"

In [15]:
graph = StateGraph(TweetState)

graph.add_node("generate", generate_tweet)
graph.add_node("evaluate", evaluate_tweet)
graph.add_node("optimize", optimize_tweet)

graph.add_edge(START, "generate")
graph.add_edge("generate", "evaluate")

graph.add_conditional_edges("evaluate", route_evaluation, {"approved": END, "needs_improvement": "optimize"})
graph.add_edge("optimize", "evaluate")

workflow = graph.compile()

workflow

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [17]:
initial_state = {
    "topic": "tgrfedre",
    "iteration": 1,
    "max_iteration": 3
}
result = workflow.invoke(initial_state)
result

KeyboardInterrupt: 